# Redes de Neuronas con conexiones residuales y entrenados según _transfer learning_
## Práctica 3

#### Hugo Fole Avellás y José Romero Conde

NOTA:

La práctica tiene que por un lado programarse y por otro ejecutarse. Ocurrieron complicaciones que imposibiitaron la ejecución en múltipes ocasiones pero creemos que el código es correcto y esta completo. No hacemos en el ejercicio 3 ahorro de memoria. 

 ### $\huge\text{Ejercicio } 1$  
 ### (_2 puntos_) Define la capa ResidualBlock (ver Figura 2), usando como base la plantilla proporcionada en la Figura 3.
 - Ten en cuenta que el número de convoluciones depende de los valores de input_channels y out-put_channels.
 - Esta red no tiene capas de Pooling. La reducción del tamaño se realiza con el parámetro strides
de las convoluciones, pero se modifica únicamente en 1 (o 2) de las convoluciones del modelo.

!['arquitectura'](ResidualBlock.png)

In [1]:
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import activations

class ResidualBlock(Model):
    def __init__(self, input_channels, output_channels, strides=(1, 1)):
        
        super().__init__()
        
        self.BN1 = layers.BatchNormalization()
        self.Conv1 = layers.Conv2D(filters = output_channels, 
                                   kernel_size = (3, 3),
                                   strides = strides,
                                   padding="same",
                                   use_bias=False)
                                   
        self.BN2 = layers.BatchNormalization()
        self.Conv2 = layers.Conv2D(filters = output_channels, 
                                   kernel_size = (3, 3),
                                   strides = (1, 1),
                                   padding="same",
                                   use_bias=False)
        
        if input_channels != output_channels:
            self.salidaDistinta = True
            self.ConvFuera = layers.Conv2D(filters = output_channels, 
                                   kernel_size = (1, 1),
                                   strides = strides,
                                   use_bias=False)
        else: self.salidaDistinta = False
            
    def call(self, x):
        x = self.BN1(x)
        y = activations.silu(x)
        x = self.Conv1(y)
        x = self.BN2(x)
        x = activations.silu(x)
        x = self.Conv2(x)
        if self.salidaDistinta:
            y = self.ConvFuera(y)
        x = x + y
        return x

 ### $\huge\text{Ejercicio } 2$  
### (_2 puntos_) Define la red ResidualNetwork (ver Figura 1). Para comprobar su correcto funcionamiento haz lo siguiente:
 - Descarga los pesos del modelo preentrenado (los podrás encontrar en el canal de Teams de la asignatura).
 - Carga los pesos en tu modelo, haciendo uso de la función proporcionada en la Figura 4.
 - Comprueba que la precisión del modelo en CIFAR-100 es superior al 69 %.

!['arquitectura'](ResidualNetwork.png)

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras import Input

ResidualNetwork = Sequential([
    Input(shape=(32,32,3)),
    layers.Conv2D(filters=16, kernel_size=(3,3),strides=(1,1),padding="same", use_bias=False), 
    # la configuración de la capa convolucional es para asegurarse que no se reduce tamaño
    ResidualBlock(16,64),
    ResidualBlock(64,64),
    ResidualBlock(64,64),
    ResidualBlock(64,128,strides=(2,2)),
    ResidualBlock(128,128),
    ResidualBlock(128,128),
    ResidualBlock(128,256,strides=(2,2)),
    ResidualBlock(256,256),
    ResidualBlock(256,256),
    layers.BatchNormalization(),
    layers.Activation(activations.silu),
    layers.GlobalAveragePooling2D(),
    layers.Dense(100,activation='softmax')
    
])

In [3]:
import pickle

def load_weights(model, weight_file):
    with open(weight_file, 'rb') as f:
        weights = pickle.load(f)

    all_vars = model.trainable_weights + model.non_trainable_weights
    weight_list = [(x, weights[x]) for x in sorted(weights.keys())]
    weights = {}
    for i, var in enumerate(all_vars):
        aux = var.path.split('/')[-2:]
        classname = '_'.join(aux[0].split('_')[:-1])
        name = aux[1]
        assigned = False
        for j, (key, value) in enumerate(weight_list):
            if classname in key and name in key:
                try:
                    all_vars[i].assign(value)
                    print(':) ',end='')
                except:
                    continue
                print('assinging', key, 'to', var.path)
                del weight_list[j]
                assigned = True
                break
        if not assigned:
            #raise Exception(var.path + ' cannot be loaded')
            print(var.path + ' cannot be loaded')

In [4]:
load_weights(ResidualNetwork, "p2_model_weights.pkl")

:) assinging conv2d_52/kernel to sequential/conv2d/kernel
:) assinging batch_normalization_38/gamma to sequential/residual_block/batch_normalization/gamma
:) assinging batch_normalization_38/beta to sequential/residual_block/batch_normalization/beta
:) assinging conv2d_54/kernel to sequential/residual_block/conv2d_1/kernel
:) assinging batch_normalization_39/gamma to sequential/residual_block/batch_normalization_1/gamma
:) assinging batch_normalization_39/beta to sequential/residual_block/batch_normalization_1/beta
:) assinging conv2d_55/kernel to sequential/residual_block/conv2d_2/kernel
:) assinging conv2d_53/kernel to sequential/residual_block/conv2d_3/kernel
:) assinging batch_normalization_40/gamma to sequential/residual_block_1/batch_normalization_2/gamma
:) assinging batch_normalization_40/beta to sequential/residual_block_1/batch_normalization_2/beta
:) assinging conv2d_56/kernel to sequential/residual_block_1/conv2d_4/kernel
:) assinging batch_normalization_41/gamma to sequent

In [5]:
## carga y procesado de datos

from tensorflow.keras.datasets import cifar100
import numpy as np

(x_train, Y_train), (x_test, Y_test) = cifar100.load_data()

x_train = (2*x_train.astype(float)-255)/(255)
x_test = (2*x_test.astype(float)-255)/(255)

# como la salida de la red son 100 nodos se sobreentiende 
# que tengo que one-hot-ear Y

y_train = np.zeros(shape=(Y_train.shape[0],max(Y_train)[0]+1))
y_train[np.arange(Y_train.size),Y_train.T] = 1

y_test = np.zeros(shape=(Y_test.shape[0],max(Y_test)[0]+1))
y_test[np.arange(Y_test.size),Y_test.T] = 1

(np.max(x_train),np.min(x_train),np.max(x_test),np.min(x_test))

(1.0, -1.0, 1.0, -1.0)

In [6]:
# comprobación de que la carga de pesos fue correcta
from random import randint

def compruebaConTest(modelo, numeroEjemplos):
    indices = [randint(0,x_test.shape[0]-1) for _ in range(numeroEjemplos)]
    y_pred = modelo(x_test[indices])
    tasaAcierto = sum([np.argmax(y_pred[i]) == np.argmax(y_test[indice]) for i, indice in enumerate(indices)])/numeroEjemplos
    print(f'    La tasa de acierto es {tasaAcierto}')
    return tasaAcierto

In [7]:
# descomentar para comprobar
compruebaConTest(ResidualNetwork, 2000);

    La tasa de acierto es 0.7095


 ### $\huge\text{Ejercicio } 3$ 
### (_4 puntos_) Entrena, mediante la técnica de fine-tuning, sobre el dataset CIFAR-10, manteniendo fijos los pesos de la red preentrenada proporcionada en el canal de Teams de la asignatura. Analiza el resultado en el conjunto de test. Se penalizarán los siguientes puntos:
 - (_-4 puntos_) No se ha entrenado la red correctamente, y no se proporciona el resultado obtenido en
el conjunto de test.
 - (_-1 punto_) No se ha utilizado la red original completa, a excepción de la última capa.
 - (_-1 punto_) No se optimizado el entrenamiento, reduciendo al máximo el consumo de memoria.
 - (_-1 punto_) No se ha hecho uso de técnicas de DataAugmentation sobre las imágenes de entrenamiento.
 - (_-2 puntos_) No se ha entrenado la red sin hacer uso de la función fit, entrenando el modelo con un
bucle de entrenamiento desde cero.

In [8]:
# primero que todo, tenemos que cargar los datos

from tensorflow.keras.datasets import cifar10
import numpy as np

(x_train, Y_train), (x_test, Y_test) = cifar10.load_data()

x_train = (2*x_train.astype(float)-255)/(255)
x_test = (2*x_test.astype(float)-255)/(255)

y_train = np.zeros(shape=(Y_train.shape[0],max(Y_train)[0]+1))
y_train[np.arange(Y_train.size),Y_train.T] = 1

y_test = np.zeros(shape=(Y_test.shape[0],max(Y_test)[0]+1))
y_test[np.arange(Y_test.size),Y_test.T] = 1

(np.max(x_train),np.min(x_train),np.max(x_test),np.min(x_test))

(1.0, -1.0, 1.0, -1.0)

In [9]:
# redefinimos el modelo

from tensorflow.keras import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RandomRotation
from tensorflow.keras.layers import RandomTranslation
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.layers import RandomContrast

# extractor de caracteristicas

aumentoDeDatos = Sequential(
    [
        RandomRotation(factor = .3), # rotaciones
        RandomTranslation(height_factor = .2, width_factor = .2), # traslaciones
        RandomFlip(), # giros
        RandomContrast(factor = .2),  # contraste
    ],
)

ResidualNetworkSinClasificador = Sequential([
    Input(shape=(32,32,3)),
    layers.Conv2D(filters=16, kernel_size=(3,3),strides=(1,1),padding="same", use_bias=False), 
    # la configuración de la capa convolucional es para asegurarse que no se reduce tamaño
    ResidualBlock(16,64),
    ResidualBlock(64,64),
    ResidualBlock(64,64),
    ResidualBlock(64,128,strides=(2,2)),
    ResidualBlock(128,128),
    ResidualBlock(128,128),
    ResidualBlock(128,256,strides=(2,2)),
    ResidualBlock(256,256),
    ResidualBlock(256,256),
    layers.BatchNormalization(),
    layers.Activation(activations.silu),
    layers.GlobalAveragePooling2D(),
    # lo unico que cambio es que la ultima capa, ahora no la tiene
    # lo hacemos así porque de esta forma todo es no entrenable
    
])
'''

load_weights(ResidualNetworkSinClasificador, "p2_model_weights.pkl")
x_train = aumentoDeDatos(x_train)
print('datos aumentados!')
#x_train = ResidualNetworkSinClasificador(x_train)
#print('características extraidas!')

'''
ResidualNetworkSinClasificador.trainable = False # que sea no entrenable

Clasificador = Sequential( # definimos el clasificador
    [
        Dropout(rate = .05),
        Dense(20, activation = 'relu'),
        Dropout(rate = .05),
        Dense(10, activation = 'softmax') # Dense con salida el número de clases del dataset
                                             # Softmax activation
    ]
)

# entrada
input = Input(shape = (32,32,3))  # capa de input
input = aumentoDeDatos(input) # le aplicamos el aumento de datos

# salida
outputs = ResidualNetworkSinClasificador(inputs = input)
outputs = Clasificador(inputs = outputs)

# modelo
modeloEjercicio3 = Model(inputs = input, outputs = outputs, name = "ejercicio3")
load_weights(modeloEjercicio3, "p2_model_weights.pkl")

sequential_3/dense_1/kernel cannot be loaded
sequential_3/dense_1/bias cannot be loaded
sequential_3/dense_2/kernel cannot be loaded
sequential_3/dense_2/bias cannot be loaded
:) assinging conv2d_52/kernel to sequential_2/conv2d_22/kernel
:) assinging batch_normalization_38/gamma to sequential_2/residual_block_9/batch_normalization_19/gamma
:) assinging batch_normalization_38/beta to sequential_2/residual_block_9/batch_normalization_19/beta
:) assinging batch_normalization_38/moving_mean to sequential_2/residual_block_9/batch_normalization_19/moving_mean
:) assinging batch_normalization_38/moving_variance to sequential_2/residual_block_9/batch_normalization_19/moving_variance
:) assinging conv2d_54/kernel to sequential_2/residual_block_9/conv2d_23/kernel
:) assinging batch_normalization_39/gamma to sequential_2/residual_block_9/batch_normalization_20/gamma
:) assinging batch_normalization_39/beta to sequential_2/residual_block_9/batch_normalization_20/beta
:) assinging batch_normalizat

In [10]:
#from numpy import expand_dims
# pasamos de los pixeles de las imagenes a las características
#x_train = [ResidualNetworkSinClasificador(expand_dims(x_train[i], 0)) for i in range(x_train.shape[0])]

In [11]:
#from tensorflow import convert_to_tensor
# pasamos de una lista de tensores a un tensor
#x_train = convert_to_tensor(x_train)

In [12]:
# ...No se ha entrenado la red sin hacer uso de la función fit, 
# entrenando el modelo con un bucle de entrenamiento desde cero...

# definimos las funciones de entrenamiento
# inspirándonos en el libro de François Chollet
from math import inf
from math import ceil as techo
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import GradientTape 
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow import reduce_mean
from time import time as tiempo

######################################################
class GeneradorBatches:

    def __init__(self, imagenes, etiquetas, tamanoBatch = 128):
        assert len(imagenes) == len(etiquetas)
        self.indice = 0
        self.imagenes = imagenes
        self.etiquetas = etiquetas
        self.tamanoBatch = tamanoBatch
        self.numBatches = techo(len(imagenes)/tamanoBatch)

    def siguiente(self):
        imagenes = self.imagenes[self.indice : self.indice + self.tamanoBatch]
        etiquetas = self.etiquetas[self.indice : self.indice + self.tamanoBatch]
        self.indice += self.tamanoBatch
        return imagenes, etiquetas

######################################################
def pasada(modelo, batchImagenes, batchEtiquetas, learning_rate=1e-3):
    
    with GradientTape() as memoria:
        predicciones = modelo(batchImagenes)
        perdidasInstancia = losses.categorical_crossentropy(batchEtiquetas, predicciones)
        perdidaMedia = reduce_mean(perdidasInstancia)
    gradientes = memoria.gradient(perdidaMedia, modelo.trainable_weights)
    optimizador = optimizers.Adam(learning_rate = learning_rate)
    optimizador.apply_gradients(zip(gradientes, modelo.trainable_weights))
    
    return perdidaMedia

######################################################
def entrena(modelo, imagenes, etiquetas, epochs, tamanoBatch, paradaTemprana='perdida', paciencilla = 15, learning_rate=1e-3):
    #diccionario para almacenar cada loss y acc de cada epoch
    hist = {
        "perdida_epoch" : [],
        "precision_epoch" : [],
        "perdida_batch" : [],
        "precision_batch" : []
    }
    
    stop = 0
    mejor_precision = 0.0
    mejor_loss = inf
    
    for epoch in range(epochs):   
        
        if stop > paciencilla :
            return hist
        
        antesEpoch = tiempo()
        print(f'Epoch: {epoch}')
        generadorBatches = GeneradorBatches(imagenes, etiquetas, tamanoBatch=tamanoBatch)
        
        for batch in range(generadorBatches.numBatches):
            
            antesBatch = tiempo()
            batchImagenes, batchEtiquetas = generadorBatches.siguiente()
            
            perdida = pasada(modelo, batchImagenes, batchEtiquetas, learning_rate=learning_rate)
            precision = compruebaConTest(modelo, 512)
            
            hist["perdida_batch" ].append(perdida.numpy())
            hist["precision_batch" ].append(precision)
            print(f'    Perdida en el batch {batch} = {perdida}')
            
            despuesBatch = tiempo()
            print(f'    Batch #{batch}/{generadorBatches.numBatches}, tardo {despuesBatch - antesBatch} segundos\n')
            
        hist["perdida_epoch"].append(hist["perdida_batch"][-1])
        hist["precision_epoch"].append(hist["precision_batch"][-1])
        
        if paradaTemprana == "precision":
            if mejor_precision < hist["precision_epoch"][-1] : 
                mejor_precision = hist["precision_epoch"][-1]
                stop = 0
            else: stop += 1 
        elif paradaTemprana == "perdida":
            if mejor_perdida > hist["perdida_epoch"][-1] : 
                mejor_precision = hist["perdida_epoch"][-1]
                stop = 0
            else: stop += 1
        despuesEpoch = tiempo()
        print(f'    Epoch #{epoch}, tardo {despuesEpoch - antesEpoch} segundos\n')
    return hist 

######################################################
def graficar(hist, batches =  True):
    if batches == True:
        fig, axes = plt.subplots(4,1, figsize = (20,20))
    else:
        fig, axes = plt.subplots(2,1, figsize = (20,20))
        
    epochs = len(hist["precision_epoch"])
    x = np.arange(1,epochs,1)
    axes[0].plot(x,hist["precision_epoch"])
    axes[0].set_title("Precisión x epochs")
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Precisión')
    axes[0].legend()

    axes[1].plot(x,hist["perdida_epoch"])
    axes[1].set_title("Perdida x epochs")
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Perdida')
    axes[1].legend()
    
    if batches == True:
        
        batches = len(hist["perdida_batch"])
        x = np.arange(1,batches,1)
        
        axes[2].plot(x,hist["precision_batch"])
        axes[2].set_title("Precisión x batches")
        axes[2].set_xlabel('Batches')
        axes[2].set_ylabel('Precisión')
        axes[2].legend()
        
        axes[3].plot(x,hist["perdida_batch"])
        axes[3].set_title("Perdida x batches")
        axes[3].set_xlabel('Batches')
        axes[3].set_ylabel('Perdida')
        axes[3].legend()
    plt.show()

In [13]:
'''
clasificador = Sequential( # definimos el clasificador
    [
        Input(shape=(256,)),
        Dropout(rate = .05),
        Dense(20, activation = 'relu'),
        Dropout(rate = .05),
        Dense(10, activation = 'softmax') # Dense con salida el número de clases del dataset
                                             # Softmax activation
    ]
)
''';

In [14]:
'''
from tensorflow.experimental.numpy import experimental_enable_numpy_behavior

experimental_enable_numpy_behavior()

x_train = x_train.reshape(50000,256)
''';

In [16]:
hist = entrena(modeloEjercicio3, x_train, y_train, epochs=2, tamanoBatch=2048,paradaTemprana = "precision", paciencilla=2, learning_rate=1e-3)

Epoch: 0
    La tasa de acierto es 0.154296875
    Perdida en el batch 0 = 2.5921428203582764
    Batch #0/25, tardo 88.16404414176941 segundos

    La tasa de acierto es 0.158203125
    Perdida en el batch 1 = 2.5395426750183105
    Batch #1/25, tardo 78.6652319431305 segundos

    La tasa de acierto es 0.162109375
    Perdida en el batch 2 = 2.4555680751800537
    Batch #2/25, tardo 77.27769613265991 segundos

    La tasa de acierto es 0.15625
    Perdida en el batch 3 = 2.41371750831604
    Batch #3/25, tardo 79.25289487838745 segundos

    La tasa de acierto es 0.197265625
    Perdida en el batch 4 = 2.349827766418457
    Batch #4/25, tardo 74.78931093215942 segundos

    La tasa de acierto es 0.19140625
    Perdida en el batch 5 = 2.2832391262054443
    Batch #5/25, tardo 110.04194188117981 segundos

    La tasa de acierto es 0.1875
    Perdida en el batch 6 = 2.2418742179870605
    Batch #6/25, tardo 97.94061279296875 segundos

    La tasa de acierto es 0.17578125
    Perdida en 

KeyboardInterrupt: 

In [ ]:
#[x_train[i].shape for i in range(x_train.shape[0])]

In [ ]:
graficar(hist=hist)

 ### $\huge\text{Ejercicio } 4$ 
### (_4 puntos_) Entrena, mediante la técnica de fine-tuning, sobre el dataset CIFAR-10, sin mantener fijos los pesos de la red preentrenada proporcionada en el canal de Teams de la asignatura. Analiza el resultado en el conjunto de test. Se penalizarán los siguientes puntos:
 - (_-4 puntos_) No se ha entrenado la red correctamente, y no se proporciona el resultado obtenido en
el conjunto de test.
 - (_-1 punto_) No se ha utilizado la red original completa, a excepción de la última capa.
 - (_-1 punto_) No se han mantenido congeladas las capas de BatchNormalization.
 - (_-1 punto_) No se ha hecho uso de un learning rate muy bajo, con el objetivo de no perder las capacidades de generalización de los pesos preentrenados
 - (_-2 puntos_) No se ha entrenado la red sin hacer uso de la función fit, entrenando el modelo con un
bucle de entrenamiento desde cero.

In [19]:
from tensorflow.keras.layers import BatchNormalization

def descongelarModelo(modelo):
    # Descongelamos las capas, dejando BatchNormalization sin entrenar
    for layer in modelo.layers:
        if not isinstance(layer, BatchNormalization): # Comprueba que la capa no sea de tipo BatchNormalization
            layer.trainable = True  

In [20]:
descongelarModelo(modeloEjercicio3)
hist2 = entrena(modeloEjercicio3, x_train, y_train, epochs=2, tamanoBatch=128,paradaTemprana = "precision", paciencilla=1, learning_rate=5e-5)

Epoch: 0
    La tasa de acierto es 0.392578125
    Perdida en el batch 0 = 1.4696695804595947
    Batch #0/391, tardo 6.009095907211304 segundos

    La tasa de acierto es 0.455078125
    Perdida en el batch 1 = 1.5754773616790771
    Batch #1/391, tardo 5.9184160232543945 segundos

    La tasa de acierto es 0.443359375
    Perdida en el batch 2 = 1.4988006353378296
    Batch #2/391, tardo 5.871199131011963 segundos

    La tasa de acierto es 0.521484375
    Perdida en el batch 3 = 1.6774628162384033
    Batch #3/391, tardo 5.832061767578125 segundos

    La tasa de acierto es 0.509765625
    Perdida en el batch 4 = 1.3627209663391113
    Batch #4/391, tardo 5.915496826171875 segundos

    La tasa de acierto es 0.515625
    Perdida en el batch 5 = 1.5889108180999756
    Batch #5/391, tardo 6.25778603553772 segundos

    La tasa de acierto es 0.51171875
    Perdida en el batch 6 = 1.4474891424179077
    Batch #6/391, tardo 6.236769914627075 segundos

    La tasa de acierto es 0.53125
  

KeyboardInterrupt: 

In [ ]:
graficar(hist=hist2)